### Sorting words

### A priori score for all words

In [10]:
import pandas
with open('data/a_priori_scores.txt', 'r') as f:
    header = f.readline().replace('#', ' ').split()
    df = pandas.read_table(f, names=header, comment='#', delimiter=r"\s+")
    df.set_index('Word', inplace=True)
df

,SumScore,AvgScore
Word,,
raise,141217,61.001
arise,147525,63.726
irate,147649,63.779
arose,152839,66.021
alter,162031,69.992
...,...,...
puppy,1799451,777.301
mamma,1829685,790.361
vivid,1878994,811.660


### Example: solving a Wordle puzzle

In [11]:
from wordle_solver import wordle_solver

In [15]:
ws = wordle_solver()

#### First attempt: using the word with the lowest a priori score

In [18]:
ws.update_information(word='raise', result='yybyb')
ws.get_compatible_word_scores().iloc[:5]

,SumScore,AvgScore
Word,,
strap,61,2.904762
scrap,73,3.476190
stray,75,3.571429
straw,79,3.761905
scram,83,3.952381


#### Second attempt: using the word with the lowest score based newly aquired knowledge

In [19]:
ws.update_information(word='strap', result='gbygb')
ws.get_compatible_word_scores().iloc[:5]

,SumScore,AvgScore
Word,,
solar,3,1.000000
sonar,3,1.000000
sugar,5,1.666667


#### Third attempt: just got lucky

In [20]:
ws.update_information(word='solar', result='ggggg')
ws.get_compatible_word_scores().iloc[:5]

,SumScore,AvgScore
Word,,
solar,1,1.0
